<a href="https://colab.research.google.com/github/itchy-squid/AdventOfCode/blob/main/2022/22/AdventOfCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#@title Model
import re

DIRECTIONS = [(1,0),(0,1),(-1,0),(0,-1)]

class Tile:
  WALL = '#'
  PATH = '.'
  VOID = ' '

class Turn:
  RIGHT = 'R'
  LEFT = 'L'

class MonkeyMap:
  def __init__(self, mm):
    self._height = len(mm)
    self._width = max([len(l) for l in mm])
    self._map = mm

  def height(self): return self._height
  def width(self): return self._width
  def starting_position(self): return (self._map[0].index(Tile.PATH),0)
  def tile(self, p): return [*(self._map[p[1]][p[0]:]),' '][0]

class Person:
  def __init__(self):
    self._d_i = 0
    self.position = (0,0)

  def direction(self): return DIRECTIONS[self._d_i]
  def turn(self, inc): self._d_i = (self._d_i + inc) % len(DIRECTIONS)

class Brain:
  def __init__(self, body, instructions):
    self._body = body
    self._instructions = instructions

  def __what_if(self, mmap, strategy, p0, d0):
    (p1,d1) = strategy.next_position(mmap, p0, d0)
    match mmap.tile(p1):
      case Tile.PATH: return (p1,d1)
      case Tile.WALL: return (p0,d0)
      case _: raise 'what is even happening right now'

  def move(self, mmap, strategy):
    #print(self._instructions)
    now = re.search(f'(\d+|{Turn.RIGHT}|{Turn.LEFT})', self._instructions).group()
    self._instructions = self._instructions[len(now):]

    match now:
      case Turn.RIGHT: self._body.turn(1)
      case Turn.LEFT: self._body.turn(-1)
      case _:
        for i in range(0,int(now)):
          self._body.position,_ = self.__what_if(
              mmap, strategy,
              self._body.position,
              self._body.direction())

    return self._instructions


In [9]:
#@title Solve
from google.colab import files
with open('2022-22-0.txt','r') as f:
  lines = f.read().splitlines()

mmap = MonkeyMap(lines[:-2])
body = Person()
brain = Brain(body, lines[-1])

def printScene (visited):
  for y in range(0,mmap.height()):
    print(''.join(['X' if (x,y) in visited
                   else mmap.tile((x,y)) for x in range(0,mmap.width())]))

def solve(strategy):
  body.position = mmap.starting_position()
  visited = {body.position}

  while brain.move(mmap, strategy):
    visited |= {body.position}
    #printScene(visited)
    #print()

  row,col=body.position[1]+1,body.position[0]+1
  return (1000*row)+(4*col)+DIRECTIONS.index(body.direction())

print(solve(Problem1Strategy()))
#print(solve(Problem2Strategy()))

6032


In [4]:
#@title Wrapping strategy for 22-1
class Problem1Strategy:
  def __add(self, mmap, p, d):
    return ((p[0]+d[0]) % mmap.width(),(p[1]+d[1]) % mmap.height())

  def next_position(self, mmap, p0, d0):
    p1 = self.__add(mmap, p0, d0)
    while mmap.tile(p1) == Tile.VOID:
      p1 = self.__add(mmap, p1, d0)
    return (p1,d0)

In [5]:
#@title Wrapping strategy for 22-2
